In [1]:
!rm -rf /kaggle/working/multimodal-eq-sizing
!git clone https://github.com/brianrp09232000/multimodal-eq-sizing.git /kaggle/working/multimodal-eq-sizing
!pip install -r requirements.txt

# Compute excess return

In [ ]:
def get_date_range(df: pd.DataFrame) -> tuple:
    grouped_by_date = df.groupby(["ticker"]).agg(['min', 'max', 'count'])["Date"]
    start = grouped_by_date["min"].min()
    end = grouped_by_date["max"].max()
    return start, end

In [ ]:
def compute_excess_return():
    df_ticker_return = get_return_data("/kaggle/input/news-trading/return_data.csv")
    start, end = get_date_range(df_ticker_return)
    df_base_ticker_return = get_single_ticker_history("SPY", start, end)
    
    return_formula = "(Close / Open) - 1"
    df_ticker_return["o2c_return"] = df_ticker_return.eval(return_formula)
    df_base_ticker_return["spy_o2c_return"] = df_base_ticker_return.eval(return_formula)
    df_base_ticker_return = df_base_ticker_return[["Date", "spy_o2c_return"]]

    excess_return_formula = "o2c_return - spy_o2c_return"
    df_with_target = pd.merge(df_ticker_return, df_base_ticker_return, on="Date", how="left")
    df_with_target["excess_return"] = df_with_target.eval(excess_return_formula)
    df_with_target = df_with_target.drop(['o2c_return', 'spy_o2c_return'], axis=1)
    
    return df_with_target

In [ ]:
df_with_target = compute_excess_return()
df_with_target.to_csv('return_data_with_target.csv', index=False)